In [2]:
import numpy as np
import random
from timeit import default_timer as timer
from datetime import datetime

In [31]:
def generate_cities(file):
    file1 = open(file, 'r')
    start = False
    cities_dict = {}

    while True:

        # Get next line from file
        line = file1.readline()

        # if line is empty
        # end of file is reached
        if not line:
            break

        if start:
            citystr, xstr, ystr = line.rstrip().split()
            city = citystr
            x = float(xstr)
            y = float(ystr)
            cities_dict[city] = [x, y]

        if line == "NODE_COORD_SECTION\n":
            start = True
    file1.close()
    return cities_dict

In [32]:
# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

In [52]:
cities_dict = generate_cities("Djibouti38.tsp")
names_list = list(cities_dict.keys())
names_list = np.array(names_list)
mutation_rate = 0.5
n_population = 100
n_cities = 38
cities_dict

{'1': [11003.6111, 42102.5],
 '2': [11108.6111, 42373.8889],
 '3': [11133.3333, 42885.8333],
 '4': [11155.8333, 42712.5],
 '5': [11183.3333, 42933.3333],
 '6': [11297.5, 42853.3333],
 '7': [11310.2778, 42929.4444],
 '8': [11416.6667, 42983.3333],
 '9': [11423.8889, 43000.2778],
 '10': [11438.3333, 42057.2222],
 '11': [11461.1111, 43252.7778],
 '12': [11485.5556, 43187.2222],
 '13': [11503.0556, 42855.2778],
 '14': [11511.3889, 42106.3889],
 '15': [11522.2222, 42841.9444],
 '16': [11569.4444, 43136.6667],
 '17': [11583.3333, 43150.0],
 '18': [11595.0, 43148.0556],
 '19': [11600.0, 43150.0],
 '20': [11690.5556, 42686.6667],
 '21': [11715.8333, 41836.1111],
 '22': [11751.1111, 42814.4444],
 '23': [11770.2778, 42651.9444],
 '24': [11785.2778, 42884.4444],
 '25': [11822.7778, 42673.6111],
 '26': [11846.9444, 42660.5556],
 '27': [11963.0556, 43290.5556],
 '28': [11973.0556, 43026.1111],
 '29': [12058.3333, 42195.5556],
 '30': [12149.4444, 42477.5],
 '31': [12286.9444, 43355.5556],
 '32': [12

In [53]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['38', '12', '36', ..., '25', '9', '2'],
       ['27', '13', '15', ..., '4', '22', '10'],
       ['4', '31', '37', ..., '3', '17', '32'],
       ...,
       ['23', '2', '18', ..., '8', '27', '24'],
       ['18', '31', '22', ..., '27', '4', '37'],
       ['31', '9', '32', ..., '16', '5', '21']], dtype='<U2')

In [54]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [55]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([27625.65034724, 24326.70428613, 29214.5580219 , 30066.50598963,
       23115.80850592, 23212.71702164, 27935.86353034, 24932.25570151,
       26661.67456702, 23837.29795075, 25337.48702249, 28436.54042975,
       26464.85003736, 24904.51838153, 27106.88620757, 26971.21735531,
       23128.47072881, 24868.16842055, 28187.82134702, 26648.23895956,
       25818.49707916, 26374.01098834, 28010.05936827, 28841.74789638,
       26848.75380928, 27094.86596428, 26240.07677304, 31207.3455451 ,
       26655.48623737, 28175.15966798, 26033.69143094, 24758.1197018 ,
       26207.57280307, 27338.4078383 , 24688.55242661, 30082.94023312,
       27191.86810849, 27892.52590549, 28783.01198647, 23998.93904711,
       24957.61040891, 24899.23763447, 26789.24224767, 23120.43470085,
       26013.37344527, 25230.51051319, 29587.79732785, 28815.34994849,
       27454.23498966, 26765.59239359, 27446.81839987, 27790.52833306,
       27455.70235398, 26522.09559913, 27032.86020958, 24988.68514512,
      

In [56]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['25', '8', '35', '7', '37', '33', '2', '1', '29', '14', '28', '20',
       '19', '9', '27', '22', '36', '12', '38', '31', '21', '3', '11',
       '13', '15', '18', '4', '26', '30', '24', '5', '23', '10', '34',
       '17', '16', '6', '32'], dtype='<U2')

In [57]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['18', '32', '37', '1', '5', '2', '16', '20', '22', '12', '17',
       '15', '30', '7', '31', '14', '10', '11', '13', '24', '8', '6',
       '33', '35', '34', '3', '27', '9', '38', '36', '4', '29', '23',
       '21', '19', '26', '25', '28'], dtype='<U2')

In [58]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['1', '16', '37', '22', '5', '2', '33', '20', '19', '4', '32', '34',
       '13', '7', '18', '14', '10', '6', '24', '11', '8', '17', '26',
       '35', '15', '3', '25', '21', '38', '36', '12', '29', '23', '9',
       '30', '31', '27', '28'], dtype='<U2')

In [61]:
best_solution = [-1,np.inf,np.array([])]
start = timer()
for i in range(500):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)
end = timer()
print("Time: {}".format(end - start))

0 22134.325091499457 26662.715685757852 15/12/22 14:50
100 21680.375397243966 26714.65876763349 15/12/22 14:50
200 22225.411767221365 27141.314815079142 15/12/22 14:50
300 23211.423271788237 27114.79925427335 15/12/22 14:50
400 22721.68482459427 27135.958502201483 15/12/22 14:51
Time: 16.74621070000012


In [63]:
best_solution[1]

19264.906481947597